#Importando as libs


In [68]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas_profiling

#Importando o dataframe Mobility Report

In [69]:
link = "https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv"
data = pd.read_csv(link)
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


#Selecionando o Brazil como região e as variáveis importantes


In [70]:
data_of_br = data.loc[data.country_region == 'Brazil',:].iloc[:,[1,2,3,7,10]].copy()
data_of_br.head()

,country_region,sub_region_1,sub_region_2,date,parks_percent_change_from_baseline
225008,Brazil,NaN,NaN,2020-02-15,-5.0
225009,Brazil,NaN,NaN,2020-02-16,-13.0
225010,Brazil,NaN,NaN,2020-02-17,-12.0
225011,Brazil,NaN,NaN,2020-02-18,-11.0
225012,Brazil,NaN,NaN,2020-02-19,-5.0


#Limpando e organizando

In [71]:
data_of_br.columns = ['country','state','city','date','parks']
data_of_br.date = pd.to_datetime(data_of_br.date)
data_of_br.index = data_of_br.date
data_of_br.drop(labels='date',axis=1,inplace=True)
data_of_br.state = data_of_br.state.str.replace("State of ","")

#Selecionando os estados nordestinos

In [72]:
states_of_northeast = ['Paraíba','Rio Grande do Norte', 'Ceará', 'Maranhão','Piauí', 'Bahia', 'Alagoas', 'Sergipe']
data_of_northeast = data_of_br.loc[data_of_br.state.isin(states_of_northeast)]

In [73]:
data_of_northeast = data_of_northeast[~data_of_northeast.state.isnull() & data_of_northeast.city.isnull() & ~data_of_northeast.parks.isnull()]

In [74]:
data_of_northeast_parks = data_of_northeast.groupby(by=[data_of_northeast.index,"state"]).mean().unstack()['parks'].rolling(window=7).mean()
data_of_northeast_parks.dropna(axis=0,how='any',inplace = True)

#Gerando a figura


In [75]:

fig, ax = plt.subplots(nrows=2,ncols=4,figsize=(14,7.5))
state = 0

for i in range(0,2):
  for ii in range(0,4):
    data_of_northeast_parks.plot(legend=False,color="grey",linewidth=1, alpha=0.4, ax=ax[i][ii])
    data_of_northeast_parks[states_of_northeast[state]].plot(legend=False,color="blue",linewidth=1.8, alpha=0.6, ax=ax[i][ii])
    
    #Ponto no inicio da pandemia
    point_ini_x = data_of_northeast_parks.loc[data_of_northeast_parks['Bahia'].values == data_of_northeast_parks['Bahia'].min()].index[0]
    point_ini_y = data_of_northeast_parks.loc[data_of_northeast_parks.index == point_ini_x][states_of_northeast[state]].values[0]

    #Ponto no fim da pandemia
    point_fin_x = data_of_northeast_parks.index[-1]
    point_fin_y = data_of_northeast_parks.loc[data_of_northeast_parks.index == point_fin_x][states_of_northeast[state]].values[0]
    
    #Tamanho do tempo do inicio da pandemia até o ultimo registro no dataframe
    arrow_x_length = dates.date2num(point_fin_x) - dates.date2num(point_ini_x)
    #Tamanho da variação do inicio da pandemia até o ultimo registro no dataframe
    arrow_y_length = abs(point_fin_y - point_ini_y)*(-1)+4
    #Indices no inicio e ultimo registro do frame
    ax[i][ii].annotate(str(int(point_ini_y)),(point_ini_x, point_ini_y),(dates.num2date(dates.date2num(point_ini_x)-20),point_ini_y-4))
    ax[i][ii].annotate(str(int(point_fin_y)),(point_fin_x, point_fin_y),((dates.num2date(dates.date2num(point_fin_x)),point_fin_y+2)))
    
    #Porcentagem de variação inicio e fim
    ax[i][ii].annotate(str(int(100-(point_fin_y*100)/point_ini_y))+'%',(point_fin_x, point_ini_y),((dates.num2date(dates.date2num(point_fin_x)-23),point_ini_y+2)), color='r',weight='bold')
    #Linhas
    ax[i][ii].arrow(point_ini_x, point_ini_y, arrow_x_length,0, ls='--', ec='black', head_width = 0.2, width=0.1)
    ax[i][ii].arrow(point_fin_x, point_fin_y, 0, arrow_y_length, ec='red', head_width = 0.2, width=0.1, ls='--', head_length=4.5)
    #Pontos inicio e fim
    ax[i][ii].scatter(point_ini_x, point_ini_y, marker='o', color='blue')
    ax[i][ii].scatter(point_fin_x, point_fin_y, marker='o', color='blue')
   
    #configurações do gráfico
    ax[i][ii].set_title(states_of_northeast[state],fontsize=12,ha='center')
    ax[i][ii].yaxis.grid(True)

    ax[i][ii].set_xlabel("")
    ax[i][ii].spines['top'].set_visible(False)
    ax[i][ii].spines['right'].set_visible(False)

    ax[i][ii].spines['left'].set_visible(False)
    ax[i][ii].set_xlim(right=dates.num2date(dates.date2num(point_fin_x)+5))
    ax[i][ii].set_xticklabels(["","Mar","Apr","May","Jun","Jul","Aug"])
    ax[i][ii].xaxis.set_tick_params(labelsize=8)
    ax[i][ii].yaxis.set_tick_params(labelsize=8)
    
    if (i==0 and ii==0 or i==0 and ii==3 or i==1 and ii==3 or i==1 and ii==0):
      ax[i][ii].yaxis.tick_right()
    else:
      ax[i][ii].set_yticklabels([])
    state += 1

plt.tight_layout()
plt.savefig("states.png",dpi=300)
plt.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  
